In [115]:
import cv2
import xml.etree.ElementTree as ET
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score, roc_curve, auc
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import random
from imageio import imread
import morphsnakes as ms
from tqdm import tqdm

In [3]:
uuid_scale={}
scale_root = ET.parse('/mnt/process/data_first/cervix_image_with_mask_and_bbox/scale_annotations.xml').getroot()
for image in scale_root.findall('image'):
    uuid = image.attrib['name'][:-4]
    for anno in image:
        if anno.tag == 'polyline':
            arr = np.array(list(map(
                lambda x: np.array(x.strip().split(','), dtype=np.float32),
                anno.attrib['points'].split(';')
            )))
            scale = arr[1][1]-arr[0][1]
            uuid_scale[uuid]=scale       

base_dir = '/mnt/process/data_first/cervix_image_with_mask_and_bbox/images'
root = ET.parse('/mnt/process/data_first/cervix_image_with_mask_and_bbox/annotations.xml').getroot()
name_df = pd.read_csv('/mnt/process/data_first/uuid_name.csv',index_col=0)
info_df = pd.read_csv('/mnt/process/data_first/yunzhou.csv',index_col=0)


cervixs=[]
labels=[]
names=[]
uuids=[]
polygons=[]
boxes=[]
img_paths=[]
scales=[]
masks = []

for image in root.findall('image'):
    flag=True
    path = image.attrib['name']
    img_path = os.path.join(base_dir, path)   
    uuid = path.split('/')[-1][:-4]  
    for anno in image:
        if anno.tag == 'box':
            xtl = round(float(anno.attrib['xtl']))
            ytl = round(float(anno.attrib['ytl']))
            xbr = round(float(anno.attrib['xbr']))
            ybr = round(float(anno.attrib['ybr']))
            box=np.array([xtl,ytl,xbr,ybr])
           
        elif anno.tag == 'polygon':
            polygon = np.array(list(map(
                lambda x: np.array(x.strip().split(','), dtype=np.float32),
                anno.attrib['points'].split(';')
            )))
            mask =  np.around(polygon).astype(np.int32)

    try:
        name = name_df.loc[uuid]['Source'] 
        name = name.split('/')[-1].split('_')[0]
    except:
        # print('Person not found')
        flag=False

    try:
        label = info_df.loc[name]['自发性早产诊断']
        # print(label)
    except:
        # print('Label not found')
        flag=False

    try:
        cervix = info_df.loc[name]['孕中期宫颈长度']
        if np.isnan(cervix):
           flag = False
        #    print(name)
    except:
        # print('Cervix not found')
        flag=False
    
    scale = uuid_scale[uuid]
    
    if flag:
        cervixs.append(cervix)
        labels.append(label)
        names.append(name)
        uuids.append(uuid)
        polygons.append(polygon)
        boxes.append(box)
        img_paths.append(img_path)
        scales.append(scale)
        masks.append(mask)
        


    

print(len(labels))
print(len(cervixs))
print(len(scales))

275
275
275


In [122]:
cnt = 0
black_areas = []
for img_path,mask,label,scale,name in zip(img_paths,masks,labels,scales,names):
    print(cnt)
    cnt += 1
    # Load the image.
    img = imread(img_path)[..., 0] / 255.0
    # g(I)
    gimg = ms.inverse_gaussian_gradient(img, alpha=1000, sigma=5.48)
    # Initialization of the level-set.
    img_tmp = cv2.imread(img_path)
    the_mask = np.zeros(img_tmp.shape[:-1], dtype=np.float32)
    cv2.fillPoly(the_mask, [mask], color=1)
    np.place(gimg, the_mask==0, np.nan)
   
  
    threshold = np.nanpercentile(gimg,40)
    print(threshold)

    res =ms.morphological_geodesic_active_contour(gimg, iterations=400,
                                             init_level_set=the_mask,
                                             smoothing=1, threshold=threshold,
                                             balloon=-1)
    black_area = (np.sum(res != 0))/scale/scale
    black_areas.append(black_area)
    # # res =  ms.morphological_chan_vese(img, iterations=50,
    # #                            init_level_set=the_mask,
    # #                            smoothing=3, lambda1=2, lambda2=1)
    fig = plt.figure(figsize=(10,10))
    fig.clf()
    ax1 = fig.add_subplot(1, 1, 1)
    ax1.imshow(img, cmap=plt.cm.gray)

    # ax2 = fig.add_subplot(1, 2, 2)
    # ax_u = ax2.imshow(np.zeros_like(img), vmin=0, vmax=1)
    ax1.contour(the_mask,[0.5],colors='b')
    ax1.contour(res, [0.5], colors='r')
    # ax_u.set_data(res)
    fig.canvas.draw()
   
    if label == 0:
        save_path = 'healthy/' + name + '.jpg'
    else:
        save_path = 'zaochan/' + name + '.jpg'
    fig.savefig(save_path)

0
0.334924763948665


/tmp/ipykernel_2903/2874355270.py:7: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  img = imread(img_path)[..., 0] / 255.0


1
0.41670240488428983
2
0.3186070518495445
3
0.3974354331309645
4
0.38923325387714747
5
0.3519184443638452
6
0.384948829691683
7
0.36516618674748746
8
0.37142820367245505
9
0.3706901641815266
10
0.35505309452214273
11
0.33450417125644966
12
0.3132026257874919
13
0.3582040053316486
14
0.3512330420909337
15
0.35778151041287165
16
0.4059380976825986
17
0.37652309557499564
18
0.43941869332728867
19
0.3553363412612658
20
0.3693934724907052


/tmp/ipykernel_2903/2874355270.py:29: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig = plt.figure(figsize=(10,10))


21
0.33165440886778164
22
0.42755735870275374
23
0.3369093493243508
24
0.3852274353659549
25
0.3733009906179857
26
0.353840137574357
27
0.3862127787011386
28
0.41856914333203815
29
0.3911819567961207


/tmp/ipykernel_2903/2874355270.py:37: UserWarning: No contour levels were found within the data range.
  ax1.contour(res, [0.5], colors='r')


30
0.39402425904735633
31
0.32715094042466075
32
0.3383268823591682
33
0.3261932722414527
34
0.37955860019403537
35
0.33962963004540814
36
0.34829313860819533
37
0.324401078659631
38
0.39398049849883465
39
0.35645527249210085
40
0.34758799789658523
41
0.34994753484464947
42
0.3549964937809204
43
0.32801510697244557
44
0.38946253918608204
45
0.4149813650577761
46
0.3058964573563201
47
0.36542220800942093
48
0.38624686617836007
49
0.3813367811541861
50
0.3513256857656163
51
0.3504258213411066
52
0.3255882199218182
53
0.36987784072903357
54
0.29809412643118277
55
0.383247829763725
56
0.3441050979814076
57
0.3571941260760109
58
0.32407683357287503
59
0.38166121543255976
60
0.40461613507219274
61
0.36193928455582813
62
0.3895128738910887
63
0.4000382805566506
64
0.3777362773254714
65
0.37295353940618003
66
0.30988788632730063
67
0.36597083976454087
68
0.4582832420943162


In [85]:
a=np.array([[1,2],[3,4]])
b=np.array([[0,0],[0,1]])
a*b

array([[0, 0],
       [0, 4]])